https://github.com/changhoonhahn/SEDflow/blob/4561ecfe3a38cc4c25df263d971a87e8a83f88ce/bin/nsa.py#L103

## Script to apply trained ANPE on the NSA data

*SET ICHUNK:*

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "12" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "12" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "12" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "12" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "12" # export NUMEXPR_NUM_THREADS=1

In [2]:
ichunk  = 9
assert ichunk < 34

In [3]:
import os, sys
import signal 
import warnings 
import numpy as np
from scipy import stats
sys.path.append('/media/SSD/Doktori/Csillagtomegbecsles_cikk/SED/SEDflow/SEDflow/src/')
from sedflow import obs as Obs
from sedflow import train as Train
import h5py

# torch
import torch
from sbi import utils as Ut
from sbi import inference as Inference

In [4]:
sample = 'toy'
itrain = 2
nhidden = 500 
nblocks = 15
print('%s training data; model %i' % (sample, itrain))

toy training data; model 2


In [18]:
from provabgs import infer as Infer
prior_sps = Infer.load_priors([
    Infer.UniformPrior(7., 12.5, label='sed'),
    Infer.FlatDirichletPrior(4, label='sed'),           # flat dirichilet priors
    Infer.UniformPrior(0., 1., label='sed'),            # burst fraction
    Infer.UniformPrior(1e-2, 13.27, label='sed'),       # tburst
    Infer.LogUniformPrior(4.5e-5, 1.5e-2, label='sed'), # log uniform priors on ZH coeff
    Infer.LogUniformPrior(4.5e-5, 1.5e-2, label='sed'), # log uniform priors on ZH coeff
    Infer.UniformPrior(0., 3., label='sed'),            # uniform priors on dust1
    Infer.UniformPrior(0., 3., label='sed'),            # uniform priors on dust2
    Infer.UniformPrior(-2., 1., label='sed')            # uniform priors on dust_index
])

In [21]:
Nsample = 100
thetas_sps  = np.array([prior_sps.transform(prior_sps.sample()) for i in range(Nsample)])

In [24]:
np.shape(thetas_sps)

(100, 12)

### Load NSA observations 
y = [u, g, r, i, z, sigma_u, sigma_g, sigma_r, sigma_i, sigma_z, z]


In [5]:
dat_dir = '/media/SSD/Doktori/Csillagtomegbecsles_cikk/SED/SEDflow/data/'
# read in NSA data with clean photometry
nsa = {}
f = h5py.File(os.path.join(dat_dir, 'nsa.photometry.hdf5'), 'r')
for k in f.keys(): 
    nsa[k] = f[k][...]
    #print(k)
f.close()

mags_nsa = np.array( [nsa['mag_u'], nsa['mag_g'], nsa['mag_r'], nsa['mag_i'], nsa['mag_z'] ] )
sigs_nsa = np.array( [nsa['sigma_u'], nsa['sigma_g'], nsa['sigma_r'], nsa['sigma_i'], nsa['sigma_z'] ] )
zred_nsa = nsa['redshift']

nsa = np.concatenate([mags_nsa.T, sigs_nsa.T, zred_nsa.reshape( np.shape(zred_nsa)[0], 1 )], axis=1)

### Set prior (this is fixed) 

In [6]:
prior_low = [7, 0., 0., 0., 0., 1e-2, np.log10(4.5e-5), np.log10(4.5e-5), 0, 0., -2.]
prior_high = [12.5, 1., 1., 1., 1., 13.27, np.log10(1.5e-2), np.log10(1.5e-2), 3., 3., 1.]
lower_bounds = torch.tensor(prior_low)
upper_bounds = torch.tensor(prior_high)
prior = Ut.BoxUniform(low=lower_bounds, high=upper_bounds, device='cpu')

### Load trained ANPE

Load test data (solely used for ANPE initialization)

In [7]:
x_test, y_test = Train.load_data('test', version=1, sample=sample, params='thetas_unt')
x_test[:,6] = np.log10(x_test[:,6])
x_test[:,7] = np.log10(x_test[:,7])

fanpe = os.path.join(Train.data_dir(), 'anpe_thetaunt_magsigz.%s.%ix%i.%i.pt' % (sample, nhidden, nblocks, itrain))

anpe = Inference.SNPE(prior=prior, 
                      density_estimator=Ut.posterior_nn('maf', hidden_features=nhidden, num_transforms=nblocks), 
                      device='cpu')
anpe.append_simulations(
    torch.as_tensor(x_test.astype(np.float32)), 
    torch.as_tensor(y_test.astype(np.float32)))

p_x_y_estimator = anpe._build_neural_net(torch.as_tensor(x_test.astype(np.float32)), torch.as_tensor(y_test.astype(np.float32)))
p_x_y_estimator.load_state_dict(torch.load(fanpe))

anpe._x_shape = Ut.x_shape_from_simulation(torch.as_tensor(y_test.astype(np.float32)))

hatp_x_y = anpe.build_posterior(p_x_y_estimator)

### Estimate posteriors

In [8]:
def get_posterior(y_nsa_i, nmcmc=10000): 
    ''' given [mag, uncertainty, redshift] of a galaxy, draw nmcmc samples of
    the posterior. 
    '''
    mcmc_anpe = hatp_x_y.sample((nmcmc,), x=torch.as_tensor(y_nsa_i), 
            show_progress_bars=False)
    return np.array(mcmc_anpe) 

class TimeoutException(Exception):   
    pass

def timeout_handler(signum, frame): 
    raise TimeoutException

In [9]:
signal.signal(signal.SIGALRM, timeout_handler)

<Handlers.SIG_DFL: 0>

In [10]:
%%time
mcmc_i = get_posterior(nsa[23456]) # with 12 cpu

CPU times: user 1min 44s, sys: 1.47 s, total: 1min 46s
Wall time: 13.3 s


In [14]:
np.shape(mcmc_i)

(10000, 11)

In [16]:
%%time
mcmc_i = get_posterior(nsa[23456]) # with 1 cpu

CPU times: user 39.1 s, sys: 192 ms, total: 39.3 s
Wall time: 39.3 s


In [10]:
%%time
_mcmc_i = get_posterior(nsa[23456]) # with 8 cpu

CPU times: user 1min 47s, sys: 208 ms, total: 1min 48s
Wall time: 13.5 s


In [10]:
mcmcs = [] 
for igal in np.arange(nsa.shape[0])[ichunk*1000:(ichunk+1)*1000]: 

    signal.alarm(300) # max 5 mins 

    try: 
        _mcmc_i = get_posterior(nsa[igal])
    except TimeoutException: 
        warnings.warn('Timed out for igal = %i' % igal) 
        _mcmc_i = np.zeros((10000, len(prior_low)))
    else:
        # Reset the alarm
        signal.alarm(0)

    mcmcs.append(_mcmc_i)
    
    if (igal % 100) == 0:
        print('Here I am: %i' % igal)


KeyboardInterrupt: 

In [10]:
np.save(fanpe.replace('.pt', '.nsa%iof34.samples.npy' % ichunk),
        np.array(mcmcs))